<a href="https://colab.research.google.com/github/Anissawen/Homework-MACS-30123/blob/main/Pset1_p1a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Q1. a

### Step1: first we try to calculate the original code time

In [ ]:
import numpy as np
import scipy.stats as sts
import time

# Set model parameters
rho = 0.5
mu = 3.0
sigma = 1.0
z_0 = mu

# Set simulation parameters
S = 1000  # Number of lives to simulate
T = 4160  # Number of periods for each simulation
np.random.seed(25)

# Draw all idiosyncratic random shocks and create containers
eps_mat = sts.norm.rvs(loc=0, scale=sigma, size=(T, S))
z_mat = np.zeros((T, S))

start_time = time.time()
for s_ind in range(S):
    z_tm1 = z_0
    for t_ind in range(T):
        e_t = eps_mat[t_ind, s_ind]
        z_t = rho * z_tm1 + (1 - rho) * mu + e_t
        z_mat[t_ind, s_ind] = z_t
        z_tm1 = z_t
end_time = time.time()
print(f"Elapsed time: {end_time - start_time:.4f} seconds")

Elapsed time: 16.3990 seconds


### Step2: we use numba.cc to precompile our codes

In [ ]:
from numba.pycc import CC
import numpy as np

cc = CC("q1a")

# Export function: x is 2D float64 (f8) arrays, and scalar inputs are int32 (i4) or float64 (f8)
@cc.export('simulate_lifetimes', 'void(f8[:,:], f8[:,:], f8, f8, f8, i4, i4)')
def simulate_lifetimes(z_mat, eps_mat, rho, mu, sigma, S, T):
    for s_ind in range(S):
        z_tm1 = mu
        for t_ind in range(T):
            e_t = eps_mat[t_ind, s_ind]
            z_t = rho * z_tm1 + (1 - rho) * mu + e_t
            z_mat[t_ind, s_ind] = z_t
            z_tm1 = z_t

cc.compile()


In [ ]:
import numpy as np
import scipy.stats as sts
import time
import q1a  # the precompiled module

# Parameters
rho = 0.5
mu = 3.0
sigma = 1.0
z_0 = mu
S = 1000  # Number of lifetimes
T = 4160  # Number of weeks (80 years × 52)

np.random.seed(25)
eps_mat = np.random.normal(loc=0, scale=sigma, size=(T, S))
z_mat = np.zeros((T, S))

# Time performance
start = time.time()
q1a.simulate_lifetimes(z_mat, eps_mat, rho, mu, sigma, S, T)
end = time.time()

print(f"AOT-compiled simulation took: {end - start:.4f} seconds")


AOT-compiled simulation took: 0.0712 seconds


# Q1.b